In [35]:
import json
import os
import re
import sqlite3
from typing import Annotated, Literal, TypedDict

import dotenv
import pandas as pd
import psycopg2
from langchain.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, MessagesState, StateGraph, add_messages
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field

dotenv.load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")
BASE_URL = os.getenv("API_BASE_URL")
# MODEL_NAME = os.getenv("MODEL_NAME")

DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "user_db",
    "user": "user",
    "password": "user",
}

## Поднимаем локальную БД

In [20]:
llm = ChatOpenAI(model="x-ai/grok-4.1-fast:free", base_url=BASE_URL, api_key=API_KEY)

In [34]:
class SubTask(TypedDict):
    id: int
    description: str
    status: Literal["pending", "in_progress", "completed", "failed"]
    result: dict | None
    error: str | None


class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    initial_task: str
    needs_decomposition: bool
    subtasks: list[SubTask]
    current_subtask_id: int | None
    planning_complete: bool
    all_tasks_complete: bool
    final_summary: dict | None

In [36]:
class RouterDecision(BaseModel):
    needs_decomposition: bool = Field(description="Требуется ли разбиение на подзадачи")
    reason: str = Field(description="Краткое объяснение решения")


def router_node(state: AgentState):
    """LLM определяет, нужна ли декомпозиция."""
    messages = state["messages"]
    task = state.get("initial_task", messages[-1].content)

    router_prompt = f"""
Определи, требует ли задача декомпозиции на подзадачи.

ЗАДАЧА: {task}

ДОСТУПНЫЕ ТАБЛИЦЫ:
- patients: id_patient, birth_date, gender, residential_area, region
- recipes: date, diagnosis_code, medication_code, id_patient
- diagnoses: diagnosis_code, name, classification
- medication: medication_code, dosage, trade_name, price, info

КРИТЕРИИ:
- Декомпозиция нужна: если требуется >1 независимых запросов или есть зависимости между запросами
- Декомпозиция НЕ нужна: если один запрос с JOIN/GROUP BY решает задачу

ПРИМЕРЫ:
- "Сколько пациентов в районе?" → false (1 запрос)
- "Топ-5 районов по обращениям" → false (1 запрос с GROUP BY)
- "Какие классы болезней в топ-5 районах?" → true (сначала топ-5, потом для каждого анализ)
"""

    llm_structured = llm.with_structured_output(RouterDecision)

    try:
        decision = llm_structured.invoke([SystemMessage(content=router_prompt)])

        print(f"Router: needs_decomposition={decision.needs_decomposition}")
        print(f"Reason: {decision.reason}")

        return {"needs_decomposition": decision.needs_decomposition}

    except Exception as e:
        print(f"Router error: {e}, falling back to heuristics")

        task_lower = task.lower()
        needs_decomp = any(
            keyword in task_lower
            for keyword in [
                "для каждого",
                "в каждом",
                "сравни",
                "в.*топ",
            ]
        )

        return {"needs_decomposition": needs_decomp}

In [38]:
class SubTaskSchema(BaseModel):
    id: int = Field(description="Уникальный ID подзадачи")
    description: str = Field(description="Описание подзадачи")
    depends_on: list[int] | None = Field(
        default=None, description="ID подзадач, от которых зависит эта (опционально)"
    )


class TaskPlan(BaseModel):
    subtasks: list[SubTaskSchema] = Field(description="Список подзадач")
    reasoning: str = Field(description="Объяснение плана декомпозиции")


def planning_node(state: AgentState):
    task = state.get("initial_task", state["messages"][-1].content)

    planning_prompt = f"""
Декомпозируй задачу пользователя на конкретные подзадачи.

ИСХОДНАЯ ЗАДАЧА: {task}

ДОСТУПНЫЕ ТАБЛИЦЫ И СВЯЗИ:
- patients: id_patient, birth_date, gender, residential_area, region
- recipes: date, diagnosis_code, medication_code, id_patient
  └─ recipes.id_patient → patients.id_patient
  └─ recipes.diagnosis_code → diagnoses.diagnosis_code
  └─ recipes.medication_code → medication.medication_code
- diagnoses: diagnosis_code, name, classification
- medication: medication_code, dosage, trade_name, price, info

ПРАВИЛА ДЕКОМПОЗИЦИИ:
1. Каждая подзадача = один SQL запрос или один анализ
2. Если нужно "топ-N", а потом "для каждого" - это 2 подзадачи
3. Если результат одной подзадачи нужен для другой - укажи зависимость в depends_on
4. Подзадачи должны быть атомарными и выполняться последовательно

ПРИМЕРЫ:

Задача: "Какие классы болезней в топ-3 районах?"
План:
- Подзадача 1: Получить топ-3 районов по количеству рецептов
- Подзадача 2: Для района из топ-1 получить распределение классов болезней (depends_on: [1])
- Подзадача 3: Для района из топ-2 получить распределение классов болезней (depends_on: [1])
- Подзадача 4: Для района из топ-3 получить распределение классов болезней (depends_on: [1])
- Подзадача 5: Агрегировать результаты и найти наиболее частые классы (depends_on: [2, 3, 4])

Задача: "Сравни средний возраст пациентов в Красносельском и Петроградском районах"
План:
- Подзадача 1: Получить средний возраст пациентов в Красносельском районе
- Подзадача 2: Получить средний возраст пациентов в Петроградском районе
- Подзадача 3: Сравнить результаты (depends_on: [1, 2])

Создай план выполнения для текущей задачи.
"""

    llm_structured = llm.with_structured_output(TaskPlan)

    try:
        plan = llm_structured.invoke([SystemMessage(content=planning_prompt)])

        print(f"Plan created with {len(plan.subtasks)} subtasks")
        print(f"Reasoning: {plan.reasoning}")

        subtasks = [
            SubTask(
                id=task.id, description=task.description, status="pending", result=None, error=None
            )
            for task in plan.subtasks
        ]

        for task in plan.subtasks:
            deps = f" (depends on: {task.depends_on})" if task.depends_on else ""
            print(f"{task.id}. {task.description} {deps}")

        return {
            "subtasks": subtasks,
            "planning_complete": True,
            "current_subtask_id": 1,
            "messages": [AIMessage(content=f"План создан:\n{plan.reasoning}")],
        }

    except Exception as e:
        print(f"Planning error: {e}")

        return {
            "subtasks": [
                SubTask(id=1, description=task, status="pending", result=None, error=None)
            ],
            "planning_complete": True,
            "current_subtask_id": 1,
            "messages": [
                AIMessage(content="Не удалось декомпозировать. Выполняю как одну задачу.")
            ],
        }

In [40]:
@tool
def execute_sql_query(query: str, limit: int = 50) -> dict:
    """
    Выполняет SQL запрос с ограничением на количество строк.

    Args:
        query: SQL запрос (только SELECT)
        limit: Максимум строк (по умолчанию 50)
    """

    # Валидация
    query_upper = query.strip().upper()
    if not query_upper.startswith("SELECT"):
        return {"success": False, "error": "Только SELECT запросы разрешены"}

    if any(keyword in query_upper for keyword in ["DROP", "DELETE", "UPDATE", "INSERT", "ALTER"]):
        return {"success": False, "error": "Опасные операции запрещены"}

    if "LIMIT" not in query_upper:
        query += f" LIMIT {limit}"

    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(query)

        columns = [desc[0] for desc in cursor.description]
        results = cursor.fetchall()
        row_count = len(results)

        conn.close()

        return {
            "success": True,
            "columns": columns,
            "data": [dict(zip(columns, row)) for row in results],
            "row_count": row_count,
            "truncated": row_count == limit,
        }
    except Exception as e:
        return {"success": False, "error": str(e)}


@tool
def get_table_schema(table_name: Literal["patients", "recipes", "diagnoses", "medication"]) -> dict:
    """
    Получает схему таблицы и примеры данных.

    Args:
        table_name: Название таблицы
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Получаем схему
        cursor.execute(f"""
            SELECT column_name, data_type, is_nullable
            FROM information_schema.columns
            WHERE table_name = '{table_name}'
            ORDER BY ordinal_position
        """)

        schema = [
            {"column": row[0], "type": row[1], "nullable": row[2] == "YES"}
            for row in cursor.fetchall()
        ]

        # Количество записей
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        total_rows = cursor.fetchone()[0]

        # Примеры данных (первые 3 строки)
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 3")
        columns = [desc[0] for desc in cursor.description]
        examples = [dict(zip(columns, row)) for row in cursor.fetchall()]

        conn.close()

        return {
            "success": True,
            "table": table_name,
            "schema": schema,
            "total_rows": total_rows,
            "examples": examples,
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

In [41]:
def execution_node(state: AgentState):
    """Выполняет текущую подзадачу"""
    current_id = state.get("current_subtask_id")
    if current_id is None:
        return {}

    subtasks = state["subtasks"]
    current_task = next((t for t in subtasks if t["id"] == current_id), None)

    if not current_task:
        return {}

    # Собираем результаты предыдущих задач для контекста
    previous_results = []
    for t in subtasks:
        if t["status"] == "completed" and t["id"] < current_id:
            previous_results.append(
                f"Задача {t['id']}: {t['description']}\n"
                f"Результат: {json.dumps(t['result'], ensure_ascii=False, indent=2)}"
            )

    context = "\n\n".join(previous_results) if previous_results else "Нет предыдущих результатов"

    # Формируем промпт для выполнения
    task_prompt = HumanMessage(
        content=f"""
ТЕКУЩАЯ ПОДЗАДАЧА: {current_task["description"]}

РЕЗУЛЬТАТЫ ПРЕДЫДУЩИХ ЗАДАЧ:
{context}

ДОСТУПНЫЕ ИНСТРУМЕНТЫ:
- execute_sql_query(query, limit): Выполняет SQL запрос
- get_table_schema(table_name): Получает схему таблицы

Выполни подзадачу, используя доступные инструменты.
Используй данные из предыдущих результатов, если они нужны.
"""
    )

    response = llm.bind_tools([execute_sql_query, get_table_schema]).invoke(
        state["messages"] + [task_prompt]
    )

    # Обновляем статус на "in_progress"
    updated_subtasks = []
    for task in subtasks:
        if task["id"] == current_id:
            task = SubTask(
                id=task["id"],
                description=task["description"],
                status="in_progress",
                result=task["result"],
                error=task["error"],
            )
        updated_subtasks.append(task)

    return {"messages": [response], "subtasks": updated_subtasks}

In [42]:
def update_status_node(state: AgentState):
    """Обновляет статус выполненной подзадачи"""
    current_id = state["current_subtask_id"]
    if current_id is None:
        return {}

    # Проверяем последнее сообщение (результат инструмента)
    last_msg = state["messages"][-1]

    # Если это не результат инструмента - пропускаем
    if not (hasattr(last_msg, "content") and isinstance(last_msg.content, str)):
        return {}

    # Парсим результат
    try:
        result = json.loads(last_msg.content)
    except:
        result = {"raw": last_msg.content}

    # Обновляем статус подзадачи
    updated_subtasks = []
    for task in state["subtasks"]:
        if task["id"] == current_id:
            task = SubTask(
                id=task["id"],
                description=task["description"],
                status="completed" if result.get("success") != False else "failed",
                result=result,
                error=result.get("error"),
            )
        updated_subtasks.append(task)

    # Определяем следующую подзадачу
    next_id = current_id + 1
    all_complete = all(t["status"] in ["completed", "failed"] for t in updated_subtasks)

    print(f"✓ Task {current_id} completed. Next: {next_id if not all_complete else 'aggregation'}")

    return {
        "subtasks": updated_subtasks,
        "current_subtask_id": None if all_complete else next_id,
        "all_tasks_complete": all_complete,
    }

In [43]:
def aggregation_node(state: AgentState):
    """Агрегирует результаты всех подзадач в финальный отчёт"""

    # Собираем все результаты
    results_summary = []
    for task in state["subtasks"]:
        if task["status"] == "completed":
            results_summary.append(
                {
                    "task_id": task["id"],
                    "description": task["description"],
                    "result": task["result"],
                }
            )

    # Формируем промпт для финального отчёта
    summary_prompt = HumanMessage(
        content=f"""
ИСХОДНЫЙ ЗАПРОС ПОЛЬЗОВАТЕЛЯ: {state["initial_task"]}

ВЫПОЛНЕННЫЕ ПОДЗАДАЧИ И ИХ РЕЗУЛЬТАТЫ:
{json.dumps(results_summary, ensure_ascii=False, indent=2)}

Создай ФИНАЛЬНЫЙ ОТЧЁТ:
1. Ответь на исходный вопрос пользователя
2. Используй данные из всех подзадач
3. Представь результат в понятном виде
4. Если есть числа - покажи их
5. Сделай краткие выводы

Формат ответа: структурированный текст с выводами.
"""
    )

    response = llm.invoke(state["messages"] + [summary_prompt])

    print("📊 Final report generated")

    return {
        "messages": [response],
        "final_summary": {
            "summary": response.content,
            "subtasks_completed": len([t for t in state["subtasks"] if t["status"] == "completed"]),
        },
    }

In [ ]:
def direct_execution_node(state: AgentState):
    """Прямое выполнение для простых задач без декомпозиции"""
    task = state["initial_task"]

    prompt = HumanMessage(
        content=f"""
ЗАДАЧА: {task}

ДОСТУПНЫЕ ТАБЛИЦЫ:
- patients: id_patient, birth_date, gender, residential_area, region
- recipes: date, diagnosis_code, medication_code, id_patient
- diagnoses: diagnosis_code, name, classification
- medication: medication_code, dosage, trade_name, price, info

ИНСТРУМЕНТЫ:
- execute_sql_query(query, limit): Выполняет SQL запрос
- get_table_schema(table_name): Получает схему таблицы

Выполни задачу используя инструменты. Ответь кратко и по существу.
"""
    )

    response = llm.bind_tools([execute_sql_query, get_table_schema]).invoke(
        state["messages"] + [prompt]
    )

    return {"messages": [response]}

In [45]:
def route_after_router(state: AgentState) -> str:
    """Куда идти после router_node"""
    if state.get("needs_decomposition"):
        return "planning"
    return "direct_execution"


def should_continue_execution(state: AgentState) -> str:
    """Продолжать выполнение подзадач?"""
    last_msg = state["messages"][-1]

    # Если есть tool_calls - идём в tools
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"

    # Если все задачи выполнены - агрегация
    if state.get("all_tasks_complete"):
        return "aggregation"

    # Продолжаем выполнение
    return "execution"


def should_continue_direct(state: AgentState) -> str:
    """Продолжать в direct режиме?"""
    last_msg = state["messages"][-1]

    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools_direct"

    return END


def after_update_status(state: AgentState) -> str:
    """После обновления статуса"""
    if state.get("all_tasks_complete"):
        return "aggregation"
    return "execution"

In [46]:
# Создаём граф
workflow = StateGraph(AgentState)

# Добавляем узлы
workflow.add_node("router", router_node)
workflow.add_node("planning", planning_node)
workflow.add_node("execution", execution_node)
workflow.add_node("tools", ToolNode([execute_sql_query, get_table_schema]))
workflow.add_node("update_status", update_status_node)
workflow.add_node("aggregation", aggregation_node)
workflow.add_node("direct_execution", direct_execution_node)
workflow.add_node("tools_direct", ToolNode([execute_sql_query, get_table_schema]))

# Точка входа
workflow.set_entry_point("router")

# Рёбра
workflow.add_conditional_edges(
    "router", route_after_router, {"planning": "planning", "direct_execution": "direct_execution"}
)

# Ветка с декомпозицией
workflow.add_edge("planning", "execution")
workflow.add_conditional_edges(
    "execution",
    should_continue_execution,
    {"tools": "tools", "execution": "execution", "aggregation": "aggregation"},
)
workflow.add_edge("tools", "update_status")
workflow.add_conditional_edges(
    "update_status", after_update_status, {"execution": "execution", "aggregation": "aggregation"}
)
workflow.add_edge("aggregation", END)

# Прямая ветка (без декомпозиции)
workflow.add_conditional_edges(
    "direct_execution", should_continue_direct, {"tools_direct": "tools_direct", END: END}
)
workflow.add_edge("tools_direct", "direct_execution")

# Компилируем
app = workflow.compile()